In [1]:
from PIL import Image
import pandas as pd
import numpy as np
import cv2 as cv
from glob import glob
from sklearn.model_selection import train_test_split
from mitosplit_net import preprocessing, augmentation, evaluation, training, plotting, util
from albumentations import Compose, Rotate, RandomRotate90, HorizontalFlip, Flip, ElasticTransform, GaussNoise, RandomCrop, Resize
from tqdm import tqdm
import os, sys
from os import path
import random as r
from import_augmentation_function import import_fun, aug_fun, import_aug_fun, normalization_fun
import matplotlib.pyplot as plt
import tensorflow as tf 

## Choice of folder and number of augmentations ##

In [2]:


files_dir = r'C:\Users\roumba\Documents\Software\deep-events\training_data'
images_dir = '220915_mtstaygold_cos7_ZEISS_fl_pos'
date, dye, cell_type, microscope, bf_fl, pos_neg = images_dir.split('_')
joined_path = os.path.join(files_dir, images_dir)
img_size = 256
data_ratio=0.1
data_split_state = None 
number_of_augmentations = 5 

#for some reason, even though it should have been deleted, the first two arrays of axis0 are zeros?# 


## Import and Split ##
                            

In [3]:

all_image_array, all_image_array_gauss = import_fun(joined_path, files_dir, images_dir)
augmentation_data, data_val, augmentation_data_gauss, data_gauss_val =  train_test_split(all_image_array, all_image_array_gauss, 
                                                                                                       test_size=data_ratio, random_state=data_split_state)
data_aug = augmentation_data
data_gauss_aug = augmentation_data_gauss
for j in range(number_of_augmentations):
    augment_data, augment_data_gauss = aug_fun(augmentation_data, augmentation_data_gauss)
    data_aug = np.concatenate((data_aug, augment_data))
    data_gauss_aug = np.concatenate((data_gauss_aug,augment_data_gauss))


100%|██████████| 166/166 [00:00<00:00, 1909.12it/s]


## Normalization and Shuffle ##

In [4]:
data_val_norm, data_aug_norm, data_gauss_val_norm, data_gauss_aug_norm = normalization_fun(data_val, data_aug, data_gauss_val, data_gauss_aug)

r.shuffle(data_val_norm)
r.shuffle(data_gauss_val_norm)
r.shuffle(data_aug_norm)
r.shuffle(data_gauss_aug_norm)

## Define GPU device ##

In [5]:
gpu = tf.config.list_physical_devices('GPU')[0]
print(gpu)
tf.config.experimental.set_memory_growth(gpu, True)
gpu = tf.device('GPU:0/') 

base_dir = r'C:\Users\roumba\Documents\Software\deep-events'
data_path = base_dir+ r'\Norm_data'
model_path = base_dir+ '\Models'

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [6]:
with gpu:
  nb_filters = 8
  firstConvSize = 9
  batch_size = [8, 16, 32, 256]
  model, history= {}, {}
  
  b=batch_size[1]
  model_name = 'ref_f%i_c%i_b%i'%(nb_filters, firstConvSize, b)
  print('Model:', model_name)
  model[model_name] = training.create_model(nb_filters, firstConvSize)
  history[model_name] = training.train_model(model[model_name], data_aug_norm, data_gauss_aug_norm, b, data_ratio)

Model: ref_f8_c9_b16
* Start Encoder Section *
* Start Center Section *
* Start Decoder Section *
Epoch 1/20
56/56 - 21s - loss: 0.9313 - binary_accuracy: 0.2192 - val_loss: 0.6911 - val_binary_accuracy: 0.6423 - 21s/epoch - 373ms/step
Epoch 2/20
56/56 - 15s - loss: 0.6382 - binary_accuracy: 0.8052 - val_loss: 0.6186 - val_binary_accuracy: 0.9732 - 15s/epoch - 269ms/step
Epoch 3/20
56/56 - 15s - loss: 0.5604 - binary_accuracy: 0.9885 - val_loss: 0.5454 - val_binary_accuracy: 0.9962 - 15s/epoch - 269ms/step
Epoch 4/20
56/56 - 15s - loss: 0.5214 - binary_accuracy: 0.9928 - val_loss: 0.4963 - val_binary_accuracy: 0.9962 - 15s/epoch - 269ms/step
Epoch 5/20
56/56 - 15s - loss: 0.4892 - binary_accuracy: 0.9926 - val_loss: 0.4540 - val_binary_accuracy: 0.9945 - 15s/epoch - 268ms/step
Epoch 6/20
56/56 - 15s - loss: 0.4566 - binary_accuracy: 0.9935 - val_loss: 0.4200 - val_binary_accuracy: 0.9952 - 15s/epoch - 269ms/step
Epoch 7/20
56/56 - 15s - loss: 0.4298 - binary_accuracy: 0.9921 - val_loss

## Saving of Data ##

In [7]:
folder_name = list(model.keys())
name1=f'\{cell_type}_{microscope}_{bf_fl}_data_val'
name2=f'\{cell_type}_{microscope}_{bf_fl}_data_gauss_val'
name3=f'\{cell_type}_{microscope}_{bf_fl}_data_aug'
name4=f'\{cell_type}_{microscope}_{bf_fl}_data_gauss_aug'

util.save_h5(data_val_norm, data_path, name1)
util.save_h5(data_gauss_val_norm, data_path, name2)
util.save_h5(data_aug_norm, data_path, name3)
util.save_h5(data_gauss_aug_norm, data_path, name4)




Saving C:\Users\roumba\Documents\Software\deep-events\Norm_data\cos7_ZEISS_fl_data_val_.h5
Done.

Saving C:\Users\roumba\Documents\Software\deep-events\Norm_data\cos7_ZEISS_fl_data_gauss_val.h5
Done.

Saving C:\Users\roumba\Documents\Software\deep-events\Norm_data\cos7_ZEISS_fl_data_aug.h5
Done.

Saving C:\Users\roumba\Documents\Software\deep-events\Norm_data\cos7_ZEISS_fl_data_gauss_aug.h5
Done.


In [9]:
import napari
viewer = napari.Viewer()
viewer.add_image(data_val_norm)


ModuleNotFoundError: No module named 'napari'

In [8]:
util.save_model(model, model_path, [f'model']*len(model), folder_name)
util.save_pkl(history, model_path, [f'history']*len(model), folder_name)


Saving C:\Users\roumba\Documents\Software\deep-events\Modelsref_f8_c9_b16/model.h5
Done.

Saving C:\Users\roumba\Documents\Software\deep-events\Modelsref_f8_c9_b16/history
Done.
